In [ ]:
import os, csv
import yfinance as yf
import pandas
def snapshot():
    with open('datasets/Book1.csv') as f:
        for line in f:
            if "," not in line:
                continue
            symbol = line.split(",")[1]
            Name = line.split(",")[0]
            #data = yf.download(symbol, start="2020-04-02", end="2021-11-6")
            data = yf.download(symbol, period="4mo", threads = True)
            data.to_csv('datasets/daily/{}.csv'.format(Name))
    return {
        "code": "success"
    }

snapshot()

In [ ]:
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# Valid periods : 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
import yfinance as yf
tok = yf.download('tatamotors.ns', period="7d", interval='1m')
#data = yf.download('shoperstop.ns', period="1d",interval = "1m", threads = True)
#close = year['Close']    
#close[-200:].mean()
#close[-50:].mean()
#tok['200ma'] = tok['Adj Close'].rolling(window=200, min_periods=0).mean()
#tok['100ma'] = tok['Adj Close'].rolling(window=100, min_periods=0).mean()
tok

In [72]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override() # <== that's all it takes :-)

# download dataframe using pandas_datareader
df = pdr.get_data_yahoo("sunpharma.NS", period="9mo",interval='1d')


[*********************100%***********************]  1 of 1 completed


In [73]:
df = df.reset_index(level='Date')

In [74]:
df.head(-1)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-06-08,673.000000,679.950012,665.099976,678.750000,671.666443,5695920
1,2021-06-09,681.049988,684.700012,667.599976,672.849976,665.828003,4492551
2,2021-06-10,673.049988,681.000000,673.000000,676.049988,668.994629,3099864
3,2021-06-11,675.200012,686.349976,674.049988,681.299988,674.189880,3822099
4,2021-06-14,682.000000,683.700012,671.549988,677.099976,670.033630,3640962
...,...,...,...,...,...,...,...
182,2022-02-28,821.000000,847.000000,821.000000,843.900024,843.900024,4123139
183,2022-03-02,844.349976,844.950012,815.700012,820.900024,820.900024,3807036
184,2022-03-03,832.799988,832.799988,817.000000,820.900024,820.900024,2657776
185,2022-03-04,816.299988,839.599976,809.349976,829.650024,829.650024,4082674


In [75]:
df['100ma'] = df['Adj Close'].rolling(window=100, min_periods=0).mean()
df['200ma'] = df['Adj Close'].rolling(window=200, min_periods=0).mean()

In [76]:
df['Date'].iloc[-1]

Timestamp('2022-03-08 00:00:00')

In [83]:
df['Adj Close'].iloc[-2]

822.6500244140625

In [84]:
df['Open'].iloc[-2]

818.0

In [85]:
last_close = df[-2:]['Adj Close'].values[0]
hma = df[-2:]['100ma'].values[0]
twoma = df[-2:]['200ma'].values[0]
per = (1/100)*last_close

In [86]:
per

8.226500244140626

In [87]:
last_close - hma

10.998241577148406

In [88]:
last_close - twoma

47.593385421018525

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=data["Date"], open=data["Open"], high=data["High"],
                low=data["Low"], close=data["Close"], name="BHARTIARTL"), 
                row=1, col=1
)

#px.line(df, x="year", y="lifeExp")

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=data['Date'], y=data['Volume'], showlegend=False), row=2, col=1)

# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
has_value = ~df["High"].isna()

import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

#df=pd.read_csv("notebooks/for_so.csv")
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_trace(
    go.Scatter(mode = 'lines',
               x=df[has_value]['Date'],
        y=df[has_value]["High"], line={'color':'black', 'width':1}
    ))

fig.add_trace(
    go.Scatter(mode = "markers",
        x=df['Date'],
        y=df["High"]
    ))

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,)

fig.show()

In [ ]:
import os, pandas
import pandas as pd
Consol = []
Break = []
def is_consolidating(df, percentage=2):
    recent_candlesticks = df[-10:]
    #print(percentage)
    max_close = recent_candlesticks['Close'].max()
    min_close = recent_candlesticks['Close'].min()

    threshold = 1 - (percentage / 100)
    if min_close > (max_close * threshold):
        return True        
    
    return False

def is_breaking_out(df, percentage=3.5):
    last_close = df[-1:]['Close'].values[0]

    if is_consolidating(df[:-1], percentage=percentage):
        recent_closes = df[-11:-1]

        if last_close > recent_closes['Close'].max():
            return True

    return False

for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))

    if is_consolidating(df, percentage=3.0):
        consol = filename + " is consolidating"
        Consol.append(consol)
    if is_breaking_out(df):
        break1 = filename + " is Breaking Out"
        Break.append(break1)
Consolidated = pd.DataFrame(Consol)
Breakout = pd.DataFrame(Break)

In [ ]:
import os, pandas
import pandas as pd
colab = []
def vol_break(df):
    recent_candlesticks = df[-15:-1]
    global Volume
    global last
    Volume = int(recent_candlesticks['Volume'].mean())
    last = df[-1:]['Volume'].values[0]
    
def breakout(Volume, last):
    if last>Volume:        
        return (Volume,last,filename)
    
for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))
     
    consol = vol_break(df)
    if breakout(Volume, last):
        colab.append(filename)
Breakout = pd.DataFrame(colab)

In [ ]:
Breakout

In [ ]:
import os, pandas
import pandas as pd
for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))

## TA LIB

In [ ]:
import pandas as pd
import pandas_ta as ta

df = pd.DataFrame() # Empty DataFrame

# Load data
#df = pd.read_csv("path/to/symbol.csv", sep=",")
# OR if you have yfinance installed
df = df.ta.ticker("aapl")

# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
#df.set_index(pd.DatetimeIndex(df["dateTime"]), inplace=True)

# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

In [ ]:
df_p = pd.DataFrame()

# Help about this, 'ta', extension
help(df_p.ta)

# List of all indicators
df_p.ta.indicators()

## Plotly candle

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')


# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=df["Date"], open=df["AAPL.Open"], high=df["AAPL.High"],
                low=df["AAPL.Low"], close=df["AAPL.Close"], name="OHLC"), 
                row=1, col=1
)

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['Date'], y=df['AAPL.Volume'], showlegend=False), row=2, col=1)

# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')


In [ ]:
df.tail()

## Twitter

In [ ]:
import streamlit as st
import datetime
date_since = st.date_input('Since', datetime.date(2021, 12, 20))



In [ ]:
from datetime import date

today = date.today()
print(today)

In [21]:
import pandas as pd
df = pd.read_csv('symb.csv')

In [22]:
list1 = df['Symbol'].tolist()
print(list1)

[nan, 'BHARTIARTL.NS', 'PRESSMN.NS', 'ABGSHIP.NS', 'BEL.NS', 'BHARATIDIL.NS', 'RDEL.NS', 'FACT.NS', 'GNFC.NS', 'ZUARIGLOB.NS', 'ZUARI.NS', 'ARIES.NS', 'GREENFIRE.NS', 'AGRITECH.NS', 'DHANUKA.NS', 'RCF.NS', 'DEEPAKFERT.NS', 'BHARATRAS.NS', 'NATHBIOGEN.NS', 'EXCELCROP.NS', 'SHARDACROP.NS', 'BAYERCROP.NS', 'GSFC.NS', 'CHAMBLFERT.NS', 'NFL.NS', 'MADRASFERT.NS', 'COROMANDEL.NS', 'UPL.NS', 'MONSANTO.NS', 'MEGH.NS', 'SPIC.NS', 'PIIND.NS', 'RALLIS.NS', 'MANGCHEFER.NS', 'INSECTICID.NS', 'CONCOR.NS', 'TCI.NS', 'ALLCARGO.NS', 'ARSHIYA.NS', 'BLUEDART.NS', 'GDL.NS', 'GATI.NS', 'SICAL.NS', 'PATINTLOG.NS', 'SNOWMAN.NS', 'NAVKARCORP.NS', 'GLOBALVECT.NS', 'GVKPIL.NS', 'NATIONALUM.NS', 'MAANALU.NS', 'CENTEXT.NS', 'HINDALCO.NS', 'TRENT.NS', 'FLFL.NS', 'JPOLYINVST.NS', 'JSWHL.NS', 'MCX.NS', 'WILLAMAGOR.NS', 'ATNINTER.NS', 'BFINVEST.NS', 'BINDALAGRO.NS', 'CREST.NS', 'NAHARCAP.NS', 'MOTILALOFS.NS', 'PNBGILTS.NS', 'TIMESGTY.NS', 'BAJAJHLDNG.NS', 'IVC.NS', 'KICL.NS', 'KIRLOSIND.NS', 'LFIC.NS', 'NSIL.NS', 'PIL